# Punto 2 Taller 13 

por: Catalina Castro, Gabriella Espitia, Isabella Parales , Juan Saldarriaga y Nestor Tabares

In [1]:
import pulp

# Sets
compuestos = ['i1', 'i2', 'i3', 'i4']
medicamentos = ['j1', 'j2']
fabricas = ['k1', 'k2']
cds = ['l1', 'l2', 'l3', 'l4']

# Parameters
precio_i = {'i1': 0, 'i2': 350, 'i3': 800, 'i4': 550}
disponibilidad_i = {'i1': 1e200, 'i2': 10000000, 'i3': 8000000, 'i4': 12000000}
demanda1_l = {'l1': 120000, 'l2': 145000, 'l3': 80000, 'l4': 35000}
demanda2_l = {'l1': 6500, 'l2': 12300, 'l3': 2600, 'l4': 800}
preciofab_j = {'j1': 2300, 'j2': 2600}

transporte_k_l = {
    ('k1', 'l1'): 4.5e-6, ('k1', 'l2'): 0.015e-6, ('k1', 'l3'): 0.8e-6, ('k1', 'l4'): 15e-6,
    ('k2', 'l1'): 65e-6, ('k2', 'l2'): 3.5e-6, ('k2', 'l3'): 3.8e-6, ('k2', 'l4'): 0.34e-6
}

# Creación PL
model = pulp.LpProblem("Punto2", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", (compuestos, medicamentos), lowBound=0, cat='Continuous')
y = pulp.LpVariable.dicts("y", (medicamentos, fabricas), lowBound=0, cat='Continuous')
z = pulp.LpVariable.dicts("z", (medicamentos, fabricas, cds), lowBound=0, cat='Continuous')
w = pulp.LpVariable("w", lowBound=0)

# Función Objetivo
model += pulp.lpSum([x[i][j] * precio_i[i] for i in compuestos for j in medicamentos]) + \
         pulp.lpSum([y[j][k] * preciofab_j[j] for j in medicamentos for k in fabricas]) + \
         pulp.lpSum([z[j][k][l] * transporte_k_l[(k, l)] for j in medicamentos for k in fabricas for l in cds])

# Restricciones
# R1: Disponibilidad
for i in compuestos:
    model += pulp.lpSum([x[i][j] for j in medicamentos]) <= disponibilidad_i[i], f"Disponibilidad_{i}"

# R2: Demanda 1
for l in cds:
    model += pulp.lpSum([z['j1'][k][l] for k in fabricas]) >= demanda1_l[l] * 5, f"Demanda1_{l}"

# R3: Demanda 2
for l in cds:
    model += pulp.lpSum([z['j2'][k][l] for k in fabricas]) >= demanda2_l[l] * 8, f"Demanda2_{l}"

# R4 to R6: Procentajes menores para j1
model += x['i1']['j1'] <= 0.35 * pulp.lpSum([x[i]['j1'] for i in compuestos]), "R4_Procentaje_menor_x11"
model += x['i2']['j1'] <= 0.15 * pulp.lpSum([x[i]['j1'] for i in compuestos]), "R5_Procentaje_menor_x21"
model += x['i3']['j1'] <= 0.10 * pulp.lpSum([x[i]['j1'] for i in compuestos]), "R6_Procentaje_menor_x31"

# R7: Función de equilibrio 1
for j in medicamentos:
    model += pulp.lpSum([x[i][j] for i in compuestos]) == pulp.lpSum([y[j][k] for k in fabricas]), f"R7_Función_equilibrio_{j}"

# R8 to R11: Procentajes menores para j2
model += x['i1']['j2'] == 0.40 * pulp.lpSum([x[i]['j2'] for i in compuestos]), "R8_Procentaje_menor_x12"
model += x['i2']['j2'] == 0.15 * pulp.lpSum([x[i]['j2'] for i in compuestos]), "R9_Procentaje_menor_x22"
model += x['i3']['j2'] == 0.10 * pulp.lpSum([x[i]['j2'] for i in compuestos]), "R10_Procentaje_menor_x32"
model += x['i4']['j2'] == 0.35 * pulp.lpSum([x[i]['j2'] for i in compuestos]), "R11_Procentaje_menor_x42"

# R12 and R13: Función de equilibrio 2 y 3
for j in medicamentos:
    model += y[j]['k1'] == pulp.lpSum([z[j]['k1'][l] for l in cds]), f"R12_Función_equilibrio_{j}_k1"
    model += y[j]['k2'] == pulp.lpSum([z[j]['k2'][l] for l in cds]), f"R13_Función_equilibrio_{j}_k2"

# Resolver problema
model.solve()

# Resultado 
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Valor de la función objetivo (w): {pulp.value(model.objective)}")

# Valores de las variables de decisión 
for v in model.variables():
    print(f"{v.name} = {v.varValue}")


Status: Optimal
Valor de la función objetivo (w): 5511730003.3446665
x_i1_j1 = 665000.0
x_i1_j2 = 71040.0
x_i2_j1 = 285000.0
x_i2_j2 = 26640.0
x_i3_j1 = 0.0
x_i3_j2 = 17760.0
x_i4_j1 = 950000.0
x_i4_j2 = 62160.0
y_j1_k1 = 1725000.0
y_j1_k2 = 175000.0
y_j2_k1 = 171200.0
y_j2_k2 = 6400.0
z_j1_k1_l1 = 600000.0
z_j1_k1_l2 = 725000.0
z_j1_k1_l3 = 400000.0
z_j1_k1_l4 = 0.0
z_j1_k2_l1 = 0.0
z_j1_k2_l2 = 0.0
z_j1_k2_l3 = 0.0
z_j1_k2_l4 = 175000.0
z_j2_k1_l1 = 52000.0
z_j2_k1_l2 = 98400.0
z_j2_k1_l3 = 20800.0
z_j2_k1_l4 = 0.0
z_j2_k2_l1 = 0.0
z_j2_k2_l2 = 0.0
z_j2_k2_l3 = 0.0
z_j2_k2_l4 = 6400.0


# Plan de distribución para minizar los costos 

## Composición de los medicamentos 

Medicamento tipo 1: 

* Compuesto tipo 1 = 665000 mg 
* Compuesto tipo 2 = 285000 mg
* Compuesto tipo 3 = No se utiliza para hacer el medicamento 1 
* Compuesto tipo 4 = 950000 mg

Medicamento tipo 2: 

* Compuesto tipo 1 = 71040 mg 
* Compuesto tipo 2 = 26540  mg
* Compuesto tipo 3 = 17760 mg
* Compuesto tipo 4 = 62160 mg

## Producción en la fábricas

Fábrica de Chia

* Produce: 1725000 mg del medicamento 1
* Produce : 171200 mg del medicamento 2 

Fábrica Cota

* Produce: 175000 mg del medicamento 1
* Produce: 6400 mg del medicamento 2

## Distribución de medicamento a los centros de distribución 

Medicamento 1: 

* Desde la fábrica de Chia

  * Aracataca: 600000 mg
  * Rionegro: 725000 mg
  * Leticia: 400000 mg 
  * Puerto Carreño: No se transporta nada del medicamento 1

* desde la fábrica de Cota

  * Aracataca: No se transporta nada del medicamento 1
  * Rionegro: No se transporta nada del medicamento 1
  * Leticia: No se transporta nada del medicamento 1
  * Puerto carreño:  175000 mg 

Medicamento 2: 

* Desde la fábrica de Chia

  * Aracataca: 52000 mg
  * Rionegro: 98400 mg
  * Leticia: 20800 mg 
  * Puerto Carreño: No se transporta nada del medicamento 1
  
* Desde la fábrica de Cota

  * Aracataca: No se transporta nada del medicamento 1
  * Rionegro: No se transporta nada del medicamento 1
  * Leticia: No se transporta nada del medicamento 1
  * Puerto carreño:  6400 mg 



# Costos Totales

El valor de la función objetivo, que representa los costos totales de producción y distribución, es 5,511,730,003.34 unidades monetarias.